### Downloading and installing the Tortoise-TTS model

In [2]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1481 (delta 364), reused 359 (delta 359), pack-reused 1073
Receiving objects: 100% (1481/1481), 53.55 MiB | 19.26 MiB/s, done.
Resolving deltas: 100% (615/615), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
  3% (3825664 of 100715777) |            | Elapsed Time: 0:00:00 ETA:  00:00:00

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


### Upload your audios samples here

In [5]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "Nik"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.mp3'), 'wb') as f:
    f.write(file_data)

Saving Audio.mp3 to Audio.mp3


In [6]:
text= "To record your voice or other audio with the screen recording, choose a microphone"
preset="fast"

In [7]:
voice_samples,conditioning_latents=load_voice(CUSTOM_VOICE_NAME)
gen=tts.tts_with_preset(text,voice_samples=voice_samples,conditioning_latents=conditioning_latents,
                        preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav',gen.squeeze(0).cpu(),24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:24<00:00,  4.02s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [00:06<00:00,  1.00s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:11<00:00,  7.07it/s]
